In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Multiple_sclerosis"
cohort = "GSE141381"

# Input paths
in_trait_dir = "../../input/GEO/Multiple_sclerosis"
in_cohort_dir = "../../input/GEO/Multiple_sclerosis/GSE141381"

# Output paths
out_data_file = "../../output/preprocess/Multiple_sclerosis/GSE141381.csv"
out_gene_data_file = "../../output/preprocess/Multiple_sclerosis/gene_data/GSE141381.csv"
out_clinical_data_file = "../../output/preprocess/Multiple_sclerosis/clinical_data/GSE141381.csv"
json_path = "../../output/preprocess/Multiple_sclerosis/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Siponimod selectively enriched regulatory T and B lymphocytes in active secondary progressive multiple sclerosis patients"
!Series_summary	"Siponimod selectively enriched regulatory T and B lymphocytes in active secondary progressive multiple sclerosis patients: 20 SPMS baseline including 3 repeats, 19 treated with 5 placebo and 14 siponimod treated."
!Series_overall_design	"20 SPMS baseline including 3 repeats (2 samples were dried out, so no data available), 19 treated with 5 placebo and 14 siponimod treated"
Sample Characteristics Dictionary:
{0: ['gender: Female', 'gender: male', 'age: unknown', 'gender: female', 'treatment: Baseline'], 1: ['age: 52', 'age: 58', 'age: 57', 'age: 35', 'age: 53', 'age: 55', 'age: 60', 'age: 47', 'age: 51', 'age: 49', 'age: 46', 'age: 44', 'treatment: Treated', 'treatment: Placebo', 'treatment: Baseline', nan], 2: ['treatment: Baseline', 'treatment: Placebo', 'treatment: Treated', nan]}


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
import pandas as pd
import numpy as np
import os
import json
from typing import Callable, Optional, Dict, Any

# Analyze the sample characteristics dictionary for data availability
sample_chars = {0: ['gender: Female', 'gender: male', 'age: unknown', 'gender: female', 'treatment: Baseline'], 
                1: ['age: 52', 'age: 58', 'age: 57', 'age: 35', 'age: 53', 'age: 55', 'age: 60', 'age: 47', 'age: 51', 'age: 49', 'age: 46', 'age: 44', 'treatment: Treated', 'treatment: Placebo', 'treatment: Baseline', np.nan], 
                2: ['treatment: Baseline', 'treatment: Placebo', 'treatment: Treated', np.nan]}

# 1. Gene Expression Data Availability
# Based on the Series title and summary, this dataset is about lymphocytes in MS patients
# It seems to contain gene expression data rather than just miRNA or methylation
is_gene_available = True  

# 2. Variable Availability and Data Type Conversion
# Trait row - The cohort is about multiple sclerosis patients
# From row 2, we can infer baseline vs treatment status
trait_row = 2  # treatment status in row 2

# Age row - Age information is available in row 1
age_row = 1

# Gender row - Gender information is available in row 0
gender_row = 0

# Define conversion functions
def convert_trait(value):
    if pd.isna(value):
        return None
    value = value.split(": ")[-1].strip().lower() if isinstance(value, str) else None
    if value == 'baseline':
        return 1  # MS patient at baseline
    elif value in ['treated', 'placebo']:
        return 0  # MS patient with treatment or placebo
    return None

def convert_age(value):
    if pd.isna(value):
        return None
    value = value.split(": ")[-1].strip() if isinstance(value, str) else None
    if value == 'unknown':
        return None
    try:
        return float(value)
    except (ValueError, TypeError):
        return None

def convert_gender(value):
    if pd.isna(value):
        return None
    value = value.split(": ")[-1].strip().lower() if isinstance(value, str) else None
    if value == 'female':
        return 0
    elif value == 'male':
        return 1
    return None

# Function to get feature data based on row index
def get_feature_data(df, row_idx, feature_name, convert_func):
    row_data = df.iloc[row_idx]
    converted_data = row_data.apply(convert_func)
    return pd.DataFrame({feature_name: converted_data}, index=df.columns)

# 3. Save Metadata
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
if trait_row is not None:
    # Load the clinical data
    clinical_data_path = os.path.join(in_cohort_dir, "clinical_data.csv")
    if os.path.exists(clinical_data_path):
        clinical_data = pd.read_csv(clinical_data_path, index_col=0)
        
        # Extract clinical features
        selected_clinical_df = geo_select_clinical_features(
            clinical_df=clinical_data,
            trait=trait,
            trait_row=trait_row,
            convert_trait=convert_trait,
            age_row=age_row,
            convert_age=convert_age,
            gender_row=gender_row,
            convert_gender=convert_gender
        )
        
        # Preview the selected clinical features
        preview = preview_df(selected_clinical_df)
        print("Preview of selected clinical features:")
        print(preview)
        
        # Save the clinical data
        os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
        selected_clinical_df.to_csv(out_clinical_data_file)
        print(f"Clinical data saved to {out_clinical_data_file}")
    else:
        print(f"Clinical data file not found at {clinical_data_path}")


Clinical data file not found at ../../input/GEO/Multiple_sclerosis/GSE141381/clinical_data.csv


### Step 3: Gene Data Extraction

In [4]:
# 1. Extract the gene expression data from the matrix file
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row IDs (gene or probe identifiers)
print("\nFirst 20 gene/probe identifiers:")
print(gene_data.index[:20])

# 3. Print the dimensions of the gene expression data
print(f"\nGene data dimensions: {gene_data.shape[0]} genes × {gene_data.shape[1]} samples")

# Note: we keep is_gene_available as True since we successfully extracted gene expression data
is_gene_available = True



First 20 gene/probe identifiers:
Index(['16650001', '16650003', '16650005', '16650007', '16650009', '16650011',
       '16650013', '16650015', '16650017', '16650019', '16650021', '16650023',
       '16650025', '16650027', '16650029', '16650031', '16650033', '16650035',
       '16650037', '16650041'],
      dtype='object', name='ID')

Gene data dimensions: 53617 genes × 40 samples


### Step 4: Gene Identifier Review

In [5]:
# These identifiers appear to be numerical IDs/probe identifiers from a microarray platform,
# not standard human gene symbols. Standard human gene symbols are typically alphanumeric
# (like BRCA1, TP53, IL6, etc.) and not purely numeric like these identifiers.
# We would need to map these probe IDs to actual gene symbols for proper analysis.

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. First get the file paths using geo_get_relevant_filepaths function
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 3. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['16657436', '16657440', '16657445', '16657447', '16657450'], 'probeset_id': ['16657436', '16657440', '16657445', '16657447', '16657450'], 'seqname': ['chr1', 'chr1', 'chr1', 'chr1', 'chr1'], 'strand': ['+', '+', '+', '+', '+'], 'start': ['12190', '29554', '69091', '160446', '317811'], 'stop': ['13639', '31109', '70008', '161525', '328581'], 'total_probes': [25.0, 28.0, 8.0, 13.0, 36.0], 'gene_assignment': ['NR_046018 // DDX11L1 // DEAD/H (Asp-Glu-Ala-Asp/His) box helicase 11 like 1 // 1p36.33 // 100287102 /// NR_034090 // DDX11L9 // DEAD/H (Asp-Glu-Ala-Asp/His) box helicase 11 like 9 // 15q26.3 // 100288486 /// NR_051985 // DDX11L9 // DEAD/H (Asp-Glu-Ala-Asp/His) box helicase 11 like 9 // 15q26.3 // 100288486 /// NR_045117 // DDX11L10 // DEAD/H (Asp-Glu-Ala-Asp/His) box helicase 11 like 10 // 16p13.3 // 100287029 /// NR_024004 // DDX11L2 // DEAD/H (Asp-Glu-Ala-Asp/His) box helicase 11 like 2 // 2q13 // 84771 /// NR_024005 // DDX11L2 // DEAD/H (Asp-Glu-A

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Determine which columns in the gene annotation dataframe contain probe IDs and gene symbols
# Looking at the gene_annotation structure, we can see:
# - 'ID' column matches the gene expression data indices (probe identifiers)
# - 'gene_assignment' column contains gene symbols and annotations

# 2. Extract the gene mapping from the annotation dataframe
# Create a mapping DataFrame with probe IDs and corresponding gene symbols
gene_mapping = get_gene_mapping(gene_annotation, prob_col='ID', gene_col='gene_assignment')

# Display the first few rows of the mapping to verify
print("Gene mapping preview (first 5 rows):")
print(gene_mapping.head())

# 3. Apply the gene mapping to convert probe-level measurements to gene expression
# The apply_gene_mapping function handles the many-to-many relationships:
# - For probes mapping to multiple genes, expression values are divided equally
# - For genes with multiple probes, the contributions are summed
gene_data = apply_gene_mapping(gene_data, gene_mapping)

# Print the shape of the resulting gene expression data
print(f"\nGene expression data shape after mapping: {gene_data.shape}")

# Preview the first few genes
print("\nFirst 10 genes after mapping:")
print(gene_data.index[:10])

# Save the gene expression data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"\nGene expression data saved to: {out_gene_data_file}")


Gene mapping preview (first 5 rows):
         ID                                               Gene
0  16657436  NR_046018 // DDX11L1 // DEAD/H (Asp-Glu-Ala-As...
1  16657440  ENST00000473358 // MIR1302-11 // microRNA 1302...
2  16657445  NM_001005484 // OR4F5 // olfactory receptor, f...
3  16657447                                                ---
4  16657450  AK302511 // LOC100132062 // uncharacterized LO...



Gene expression data shape after mapping: (81076, 40)

First 10 genes after mapping:
Index(['A-', 'A-2', 'A-52', 'A-E', 'A-I', 'A-II', 'A-IV', 'A-V', 'A0', 'A1'], dtype='object', name='Gene')



Gene expression data saved to: ../../output/preprocess/Multiple_sclerosis/gene_data/GSE141381.csv


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the index of gene expression data
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Gene data shape after normalization: {normalized_gene_data.shape}")
print(f"First 5 gene symbols after normalization: {normalized_gene_data.index[:5]}")

# Save the normalized gene data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# 2. Check if trait is available and clinical data exists
is_trait_available = trait_row is not None
linked_data = None

if is_trait_available:
    # Create the clinical data here if it hasn't been done yet
    # This is needed because in previous steps we couldn't save the clinical data
    clinical_data = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Save the clinical data
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    clinical_data.to_csv(out_clinical_data_file)
    print(f"Clinical data saved to {out_clinical_data_file}")
    
    # Link clinical and genetic data
    linked_data = geo_link_clinical_genetic_data(clinical_data, normalized_gene_data)
    print(f"Linked data shape before handling missing values: {linked_data.shape}")
    
    # 3. Handle missing values
    linked_data = handle_missing_values(linked_data, trait)
    print(f"Data shape after handling missing values: {linked_data.shape}")
    
    # 4. Determine if trait and demographic features are biased
    is_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)
    print(f"Data shape after removing biased features: {linked_data.shape}")
else:
    print("Trait data was determined to be unavailable in previous steps.")
    is_biased = True  # Set to True since we can't evaluate without trait data
    linked_data = normalized_gene_data.T  # Just use gene expression data
    print(f"Using only gene expression data, shape: {linked_data.shape}")

# 5. Validate and save cohort info
is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=True,
    is_trait_available=is_trait_available,
    is_biased=is_biased,
    df=linked_data,
    note="Dataset contains gene expression data from multiple sclerosis patients, with treatment status used as the trait."
)

# 6. Save linked data if usable
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print("Dataset deemed not usable for associational studies.")

Gene data shape after normalization: (23274, 40)
First 5 gene symbols after normalization: Index(['A1BG', 'A1BG-AS1', 'A1CF', 'A2M', 'A2M-AS1'], dtype='object', name='Gene')


Normalized gene data saved to ../../output/preprocess/Multiple_sclerosis/gene_data/GSE141381.csv
Clinical data saved to ../../output/preprocess/Multiple_sclerosis/clinical_data/GSE141381.csv
Linked data shape before handling missing values: (40, 23277)


Data shape after handling missing values: (25, 23277)
For the feature 'Multiple_sclerosis', the least common label is '0.0' with 6 occurrences. This represents 24.00% of the dataset.
The distribution of the feature 'Multiple_sclerosis' in this dataset is fine.

Quartiles for 'Age':
  25%: 49.0
  50% (Median): 52.0
  75%: 57.0
Min: 35.0
Max: 60.0
The distribution of the feature 'Age' in this dataset is fine.

For the feature 'Gender', the least common label is '1.0' with 5 occurrences. This represents 20.00% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.

Data shape after removing biased features: (25, 23277)


Linked data saved to ../../output/preprocess/Multiple_sclerosis/GSE141381.csv
